In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
from ROOT import larutil
from larcv import larcv
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np

In [ ]:
# LOAD THE FILE
output_files = ["test.root"]
iolcv = larcv.IOManager( larcv.IOManager.kREAD )
for f in output_files:
    iolcv.add_in_file( f )
iolcv.initialize()
nentries = iolcv.get_n_entries()

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
detid = larlite.geo.kICARUS
#detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# define a function that will turn our image2d into a lardly plot
# modified from lardly.data.visualize_larcv_image2d

def visualize_larcv_image2d( image2d, minz=0.0, maxz=200.0, reverse_ticks=False, uvplane_maxcols=2400 ):
    meta = image2d.meta()
    imgnp = np.transpose( larcv.as_ndarray( image2d ), (1,0) )
    if uvplane_maxcols is not None and meta.plane() in [0,1]:
        imgnp = imgnp[:,0:uvplane_maxcols]
        maxx = uvplane_maxcols
    else:
        maxx = meta.max_x()
    #print("image shape: ",imgnp.shape," maxx=",maxx)
    xaxis = np.linspace( meta.min_x(), maxx, endpoint=False, num=int(maxx/meta.pixel_width()) )
    yaxis = np.linspace( meta.min_y(), meta.max_y(), endpoint=False, num=meta.rows() )
    #print(type(imgnp),type(xaxis),type(yaxis))

    imgnp[ imgnp<minz ] = 0
    imgnp[ imgnp>maxz ] = maxz

    if reverse_ticks:
        imgnp = np.flip( imgnp, axis=0 )

    heatmap = {
        #"type":"heatmapgl",                                                                                                                                                                  
        "type":"heatmap",
        "z":imgnp,
        "x":xaxis,
        "y":yaxis,
        "colorscale":"Jet",
        }
    return heatmap


In [ ]:
# Get entry data
ENTRY = 0
iolcv.read_entry(ENTRY)

ev_spimg = iolcv.get_data(larcv.kProductSparseImage,'sparsessnet')
ev_input = iolcv.get_data(larcv.kProductImage2D,'ssnetinput')
ssnet_v = []
input_v = []
for i in range(ev_spimg.SparseImageArray().size()):
    spimg = ev_spimg.SparseImageArray().at(i)
    print("spimg[",i,"] len=",spimg.len())
    if spimg.len()==0:
        continue
    nelems = spimg.len()
    ssnet = larcv.Image2D( spimg.meta(0) )
    stride = spimg.stride()
    print("spimg[",i,"] meta=",ssnet.meta().dump())
    for ipix in range(nelems):
        maxclass = 0
        maxscore = 0
        showerscore = 0
        trackscore  = 0
        for iclass in range(spimg.nfeatures()):
            score = spimg.pixellist().at( ipix*stride + 2 + iclass )
            if iclass<=1:
                trackscore  += score
            else:
                showerscore += score
            if score>maxscore:
                maxclass = iclass
                maxscore = score
        irow = int(spimg.pixellist().at( ipix*stride ))
        icol = int(spimg.pixellist().at( ipix*stride+1))
        #print(irow,icol)
        if showerscore>trackscore:
            # LABEL SHOWER WITH 1
            ssnet.set_pixel(irow,icol,1)
        else:
            # LABEL TRACK WITH 2
            ssnet.set_pixel(irow,icol,2)
    ssnet_v.append(ssnet) 
    input_v.append( ev_input.as_vector()[i] )

In [ ]:
IMG_INDEX = 5
fig1 = go.Figure(data=[visualize_larcv_image2d( ssnet_v[IMG_INDEX], reverse_ticks=False )],
                layout={"height":800})
fig1.show()
fig2 = go.Figure(data=[visualize_larcv_image2d( input_v[IMG_INDEX], reverse_ticks=False )],
                layout={"height":800})
fig2.show()